### Updating packages for plotly

In [ ]:
!pip install --upgrade plotly
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install us

     |████████████████████████████████| 13.2MB 328kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 972kB 8.9MB/s 
     |████████████████████████████████| 14.8MB 349kB/s 
     |████████████████████████████████| 6.5MB 41.5MB/s 
     |████████████████████████████████| 225kB 8.9MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=87c4e25e429608efde89db936eb5b36db137cdca79eabcee418a8a5f5ea6458c
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built pyshp
Requirement already up-to-date: shapely in /usr/local/lib/python3.6/dist-packages (1.7.1)
     |████████████████████████████████| 133kB 8.7MB/s 
  Created wheel for us: filename=us-2.0.2-cp36-none-any.whl size=11930 sha256=280bb71a2d15c6cdff4a9a741fe6d545061b52823966dc4710d1096a238fabdd
  Stored in directory: /root/.cach

### Setup

In [ ]:
from tqdm import tqdm
from datetime import datetime
from google.colab import files, drive
import json
import os
drive.mount('/content/drive')

Mounted at /content/drive


Defining folder and file paths

In [ ]:
# Defining google drive paths
interim_path = os.path.join('drive','My Drive', 'Colab Notebooks','Interim')
historical_path = os.path.join('drive','My Drive', 'samples_election_filter_2016.jsonl')
historical_updated_path = os.path.join(interim_path, 'historical_updated.jsonl')
updated_path = os.path.join(interim_path, 'updated.jsonl')
print(os.listdir(interim_path))
clean_path = os.path.join(interim_path, 'cleaned.jsonl')

['collected.jsonl', 'cleaned.jsonl', 'updated.jsonl', 'historical_updated.jsonl']


In [ ]:
def read_jsonl(filename):
    '''Iterates through a JSONL file'''
    with open(filename, "r", encoding="utf8") as f:
        for line in f:
            yield json.loads(line.rstrip('\n|\r'))

### (Test) Reading tweet data from json file

In [ ]:
datareader = read_jsonl(clean_path)

In [ ]:
next(datareader)

{'acct_desc': 'i thank You God for this most amazing \nday: for the leaping greenly spirits of trees\nand a blue true dream of sky; and for everything',
 'favorite_count': 0,
 'followers': 103,
 'following': 18,
 'hashtags': 'biden,campaign,joe,fracking,fossilfuel,jobs,china,globalgraft,thebigguy,ukraine,biden,comeonman,bobulinski,kickback,russiahoax,trump,wsj,pennsylvania,ohio,michigan,texas,newmexico,florida,virginia,bidenharris2020landslide,chuckle',
 'latitude': '',
 'location': 'California',
 'longitude': '',
 'retweet_count': 0,
 'standardized_loc': 'California',
 'text': 'biden campaign putting the lid on joe fracking fossilfuel jobs china globalgraft thebigguy ukraine biden comeonman bobulinski kickback russiahoax trump wsj pennsylvania ohio michigan texas newmexico florida virginia bidenharris2020landslide chuckle',
 'tweet_created_ts': '2020-10-25 23:59:56',
 'user_created_ts': '2010-05-14 19:52:49',
 'user_total_tweets': 80,
 'username': '2haveandhavenot'}

In [ ]:
for _ in range(10):
  print(next(datareader)['text'] + '\n')

###Getting Sentiment using WordBlob

In [ ]:
from textblob import TextBlob
from collections import defaultdict


In [ ]:
polarity_counter = defaultdict(list)
subjectivity_counter = defaultdict(list)
updated_tweets = []
for tweet in tqdm(read_jsonl(clean_path)):
  text = tweet['text']
  hashtags = tweet['hashtags']

  blob = TextBlob(text)
  tweet['blob_sentiment'] = blob.sentiment.polarity
  tweet['blob_subjectivity'] = blob.sentiment.subjectivity
  updated_tweets.append(tweet)

  if 'trump' in hashtags and 'biden' not in hashtags:
    polarity_counter['trump'].append(blob.sentiment.polarity)
    subjectivity_counter['trump'].append(blob.sentiment.subjectivity)
  elif  'biden' in hashtags and 'trump' not in hashtags:
    polarity_counter['biden'].append(blob.sentiment.polarity)
    subjectivity_counter['biden'].append(blob.sentiment.subjectivity)
  elif 'trump' in hashtags and 'biden' in hashtags: 
    polarity_counter['trump+biden'].append(blob.sentiment.polarity)
    subjectivity_counter['trump+biden'].append(blob.sentiment.subjectivity)
  

NameError: ignored

In [ ]:
trump_mean_polarity = sum(polarity_counter['trump'])/len(polarity_counter['trump'])
biden_mean_polarity = sum(polarity_counter['biden'])/len(polarity_counter['biden'])
trumpbiden_mean_polarity = sum(polarity_counter['trump+biden'])/len(polarity_counter['trump+biden'])

trump_mean_subjectivity = sum(subjectivity_counter['trump'])/len(subjectivity_counter['trump'])
biden_mean_subjectivity = sum(subjectivity_counter['biden'])/len(subjectivity_counter['biden'])
trumpbiden_mean_subjectivity = sum(subjectivity_counter['trump+biden'])/len(subjectivity_counter['trump+biden'])

### Plotting WordBlob sentiment between different hashtags

In [ ]:
import matplotlib.pyplot as plt

####Mean polarity scores

In [ ]:
polarity = [trump_mean_polarity, biden_mean_polarity, trumpbiden_mean_polarity]
x = ['Trump', 'Biden', 'Trump + Biden']
x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, polarity)
plt.xlabel('Candidate Hashtag')
plt.ylabel('Average Polarity')
plt.title('Average Polarity of Tweets with Given Candidate Hashtags')

plt.xticks(x_pos, x)
plt.show()

####Mean subjectivity scores

In [ ]:
subjectivity = [trump_mean_subjectivity, biden_mean_subjectivity, trumpbiden_mean_subjectivity]
x = ['Trump', 'Biden', 'Trump + Biden']
x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, polarity)
plt.xlabel('Candidate Hashtag')
plt.ylabel('Average Subjectivity')
plt.title('Average Subjectivity of Tweets with Given Candidate Hashtags')

plt.xticks(x_pos, x)
plt.show()

In [ ]:
updated_tweets[:5]

### Getting Sentiment based on Flair



In [ ]:
!pip3 install flair

     |████████████████████████████████| 450kB 12.3MB/s 
     |████████████████████████████████| 1.3MB 29.5MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 983kB 45.1MB/s 
     |████████████████████████████████| 798kB 43.1MB/s 
     |████████████████████████████████| 19.7MB 3.8MB/s 
     |████████████████████████████████| 1.1MB 33.6MB/s 
     |████████████████████████████████| 890kB 41.5MB/s 
     |████████████████████████████████| 2.9MB 26.7MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14377 sha256=1e6fb0024726b0b4e7b3e540be53008bd5b3831945cf65505da4610218094c3d
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15e92f0d46c5e464d8e4b9b
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25021 sha256=7ef8b64ff3e03994db45db1c2b5e2a9e2a3db69b8388d7c7f53f8cfc9687ad3b
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b57

In [ ]:
import torch
import flair

In [ ]:
torch.cuda.is_available()

False

In [ ]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2020-12-07 11:59:15,099 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_3.1.pt not found in cache, downloading to /tmp/tmpvb8ulcs3


100%|██████████| 266147697/266147697 [01:04<00:00, 4110981.48B/s]

2020-12-07 12:00:20,213 copying /tmp/tmpvb8ulcs3 to cache at /root/.flair/models/sentiment-en-mix-distillbert_3.1.pt


2020-12-07 12:00:20,682 removing temp file /tmp/tmpvb8ulcs3
2020-12-07 12:00:20,718 loading file /root/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [ ]:
s = flair.data.Sentence('orange man is gone, not bad')
flair_sentiment.predict(s)
s.labels

[POSITIVE (0.9444)]

In [ ]:
flair_sent = []
counter = 1
for tweet in updated_tweets:
  if counter % 100 == 0:
    print(counter)
  counter += 1
  s = flair.data.Sentence(tweet['text'])
  flair_sentiment.predict(s)
  total_sentiment = s.labels
  tweet['flair_sentiment'] = total_sentiment
  flair_sent.append(tweet)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [ ]:
with open(updated_path, 'w') as f:
  for tweet in flair_sent:
    tweet['flair_sentiment'] = str(tweet['flair_sentiment'])
    json.dump(tweet, f)
    f.write('\n')


###Grouping and Mapping Sentiment Scores by State

Using the US library to convert state names to FIPS ids

In [ ]:
import us
import re
from collections import defaultdict
import plotly
import plotly.express as px
from plotly.offline import plot

ModuleNotFoundError: ignored

In [ ]:
state_sentiment_pos = defaultdict(int)
state_sentiment_neg = defaultdict(int)
state_sentiment_total = defaultdict(int)

#### Getting general election sentiment

In [ ]:
# testing wordblob
for tweet in tqdm(read_jsonl(updated_path)):
  state_abbr = us.states.lookup(tweet['standardized_loc']).abbr

  state_sentiment_total[state_abbr] += 1
  if tweet['blob_sentiment'] > 0:
    state_sentiment_pos[state_abbr] += 1
  else:
    state_sentiment_neg[state_abbr] += 1

# normalize data
state_sentiment_norm = {k: v/state_sentiment_total[k] for k, v in state_sentiment_pos.items()}

# wrangle our data into correct format
tup_list = [(k, v) for k, v in state_sentiment_norm.items()]
tup_list = sorted(tup_list, key = lambda x: x[0])
abbr, pos_count = zip(*tup_list)

# get diff between state sent and twitter sentiment averages
avgstatewide_sentiment = sum(pos_count)/len(pos_count)
avgtwitter_sentiment = sum(state_sentiment_pos.values())/(sum(state_sentiment_pos.values())+sum(state_sentiment_neg.values()))
print(avgstatewide_sentiment, avgtwitter_sentiment)

fig = px.choropleth(locations=abbr, locationmode="USA-states", color=pos_count, scope="usa", labels = {'color': '% tweets positive'}, title='Percentage of Tweets Mentioning Biden that are Positive')
fig.show()

89644it [00:02, 35503.57it/s]


0.4784224230444996 0.4771094551782607


In [ ]:
# can change between historical and current
for tweet in tqdm(read_jsonl(historical_updated_path)):
  state_abbr = us.states.lookup(tweet['standardized_loc']).abbr

  state_sentiment_total[state_abbr] += 1
  if 'POSITIVE' in tweet['flair_sentiment']:
    state_sentiment_pos[state_abbr] += 1
  elif 'NEGATIVE' in tweet['flair_sentiment']:
    state_sentiment_neg[state_abbr] += 1
  else:
    print('what is going on here')
  

2185359it [00:23, 93631.54it/s] 


#### Filtering for only Biden tweets

In [ ]:
state_sentiment_pos = defaultdict(int)
state_sentiment_neg = defaultdict(int)
state_sentiment_total = defaultdict(int)

for tweet in tqdm(read_jsonl(updated_path)):
  state_abbr = us.states.lookup(tweet['standardized_loc']).abbr
  text = tweet['text'].lower()
  if 'trump' in text or 'biden' not in text:
    continue

  state_sentiment_total[state_abbr] += 1
  if 'POSITIVE' in tweet['flair_sentiment']:
    state_sentiment_pos[state_abbr] += 1
  elif 'NEGATIVE' in tweet['flair_sentiment']:
    state_sentiment_neg[state_abbr] += 1
  else:
    print('what is going on here')

# normalize data
state_sentiment_norm = {k: v/state_sentiment_total[k] for k, v in state_sentiment_pos.items()}

# wrangle our data into correct format
tup_list = [(k, v) for k, v in state_sentiment_norm.items()]
tup_list = sorted(tup_list, key = lambda x: x[0])
abbr, pos_count = zip(*tup_list)

# get diff between state sent and twitter sentiment averages
avgstatewide_sentiment = sum(pos_count)/len(pos_count)
avgtwitter_sentiment = sum(state_sentiment_pos.values())/(sum(state_sentiment_pos.values())+sum(state_sentiment_neg.values()))
print(avgstatewide_sentiment, avgtwitter_sentiment)


0it [00:00, ?it/s]
1it [00:02,  2.07s/it]
5184it [00:03,  1.45s/it]
7035it [00:03,  1.02s/it]
10180it [00:04,  1.41it/s]
15677it [00:04,  2.01it/s]
21038it [00:04,  2.87it/s]
27338it [00:04,  4.10it/s]
33590it [00:04,  5.86it/s]
38800it [00:04,  8.37it/s]
43725it [00:04, 11.95it/s]
49969it [00:04, 17.07it/s]
55955it [00:04, 24.39it/s]
62303it [00:05, 34.83it/s]
67818it [00:05, 49.74it/s]
73671it [00:05, 71.04it/s]
79700it [00:05, 101.43it/s]
89644it [00:05, 16319.57it/s]


Visualizing sentiment on US map

In [ ]:
fig = px.choropleth(locations=abbr, locationmode="USA-states", color=pos_count, scope="usa", labels = {'color': '% tweets positive'}, title='Percentage of Tweets Mentioning Biden that are Positive')
fig.show()

#### Filtering for only Trump tweets

In [ ]:
state_sentiment_pos = defaultdict(int)
state_sentiment_neg = defaultdict(int)
state_sentiment_total = defaultdict(int)

for tweet in tqdm(read_jsonl(updated_path)):
  state_abbr = us.states.lookup(tweet['standardized_loc']).abbr
  text = tweet['text'].lower()
  if 'biden' in text or 'trump' not in text:
    continue

  state_sentiment_total[state_abbr] += 1
  if 'POSITIVE' in tweet['flair_sentiment']:
    state_sentiment_pos[state_abbr] += 1
  elif 'NEGATIVE' in tweet['flair_sentiment']:
    state_sentiment_neg[state_abbr] += 1
  else:
    print('what is going on here')

# normalize data
state_sentiment_norm = {k: v/state_sentiment_total[k] for k, v in state_sentiment_pos.items()}

# wrangle our data into correct format
tup_list = [(k, v) for k, v in state_sentiment_norm.items()]
tup_list = sorted(tup_list, key = lambda x: x[0])
abbr, pos_count = zip(*tup_list)

# get diff between state sent and twitter sentiment averages
avgstatewide_sentiment = sum(pos_count)/len(pos_count)
avgtwitter_sentiment = sum(state_sentiment_pos.values())/(sum(state_sentiment_pos.values())+sum(state_sentiment_neg.values()))
print(avgstatewide_sentiment, avgtwitter_sentiment)


0it [00:00, ?it/s]
5521it [00:00, 55106.39it/s]
10968it [00:00, 54913.38it/s]
16474it [00:00, 54956.55it/s]
21909it [00:00, 54772.16it/s]
27513it [00:00, 55143.28it/s]
33166it [00:00, 55539.27it/s]
38804it [00:00, 55785.98it/s]
44184it [00:00, 55159.68it/s]
49706it [00:00, 55176.79it/s]
55003it [00:01, 52618.34it/s]
60483it [00:01, 53254.25it/s]
66069it [00:01, 54009.35it/s]
71705it [00:01, 54692.08it/s]
77135it [00:01, 54570.58it/s]
82904it [00:01, 55470.00it/s]
89644it [00:01, 54774.69it/s]

0.21784342807012308 0.21421301344965468


In [ ]:
fig_trump = px.choropleth(locations=abbr, locationmode="USA-states", color=pos_count, scope="usa", labels = {'color': '% tweets positive'}, title='Percentage of Tweets Mentioning Trump that are Positive', )
fig_trump.show()

### Filtering for only Hillary Tweets

In [ ]:
state_sentiment_pos = defaultdict(int)
state_sentiment_neg = defaultdict(int)
state_sentiment_total = defaultdict(int)

for tweet in tqdm(read_jsonl(historical_updated_path)):
  state_abbr = us.states.lookup(tweet['standardized_loc']).abbr
  text = tweet['text'].lower()
  if 'trump' in text or 'hillary' not in text or 'clinton' not in text:
    continue

  state_sentiment_total[state_abbr] += 1
  if 'POSITIVE' in tweet['flair_sentiment']:
    state_sentiment_pos[state_abbr] += 1
  elif 'NEGATIVE' in tweet['flair_sentiment']:
    state_sentiment_neg[state_abbr] += 1
  else:
    print('what is going on here')

# normalize data
state_sentiment_norm = {k: v/state_sentiment_total[k] for k, v in state_sentiment_pos.items() if v/state_sentiment_total[k] <.95}

# wrangle our data into correct format
tup_list = [(k, v) for k, v in state_sentiment_norm.items()]
tup_list = sorted(tup_list, key = lambda x: x[0])
abbr, pos_count = zip(*tup_list)

# get diff between state sent and twitter sentiment averages
avgstatewide_sentiment = sum(pos_count)/len(pos_count)
avgtwitter_sentiment = sum(state_sentiment_pos.values())/(sum(state_sentiment_pos.values())+sum(state_sentiment_neg.values()))
print(avgstatewide_sentiment, avgtwitter_sentiment)

fig_hillary = px.choropleth(locations=abbr, locationmode="USA-states", color=pos_count,\
                            scope="usa", labels = {'color': '% tweets positive'}, \
                            title='Percentage of 2016 Tweets Mentioning Hillary that are Positive')
fig_hillary.show()

2185359it [00:19, 113225.60it/s]


0.19501881783804692 0.19051743566854773


### Filtering for only 2016 tweets mentioning Trump

In [ ]:
state_sentiment_pos = defaultdict(int)
state_sentiment_neg = defaultdict(int)
state_sentiment_total = defaultdict(int)

for tweet in tqdm(read_jsonl(historical_updated_path)):
  state_abbr = us.states.lookup(tweet['standardized_loc']).abbr
  text = tweet['text'].lower()
  if 'hillary' in text or 'trump' not in text or 'clinton' in text:
    continue

  state_sentiment_total[state_abbr] += 1
  if 'POSITIVE' in tweet['flair_sentiment']:
    state_sentiment_pos[state_abbr] += 1
  elif 'NEGATIVE' in tweet['flair_sentiment']:
    state_sentiment_neg[state_abbr] += 1
  else:
    print('what is going on here')

# normalize data
state_sentiment_norm = {k: v/state_sentiment_total[k] for k, v in state_sentiment_pos.items()}

# wrangle our data into correct format
tup_list = [(k, v) for k, v in state_sentiment_norm.items()]
tup_list = sorted(tup_list, key = lambda x: x[0])
abbr, pos_count = zip(*tup_list)

# get diff between state sent and twitter sentiment averages
avgstatewide_sentiment = sum(pos_count)/len(pos_count)
avgtwitter_sentiment = sum(state_sentiment_pos.values())/(sum(state_sentiment_pos.values())+sum(state_sentiment_neg.values()))
print(avgstatewide_sentiment, avgtwitter_sentiment)

fig_trump_2016 = px.choropleth(locations=abbr, locationmode="USA-states", color=pos_count,\
                            scope="usa", labels = {'color': '% tweets positive'}, \
                            title='Percentage of 2016 Tweets Mentioning Trump that are Positive')
fig_trump_2016.show()

2185359it [00:23, 91772.84it/s] 

0.22911887879203738 0.22726438523984535
